Importing libraries

In [ ]:
import numpy as np 
import pandas as pd 
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

Website Scraping

In [ ]:
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})
df=pd.read_html(str(indiatable))
# convert list to dataframe
df=pd.DataFrame(df[0])
df

Ignore cells with a borough that is Not assigned. 

In [ ]:
df = df[df.Borough != "Not assigned"]

More than one neighborhood can exist in one postal code area. Grouping by PostalCode

In [ ]:
df.groupby(["Postal Code", "Borough"], as_index=False)["Neighbourhood"]

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [ ]:
df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df.Borough

In [ ]:
df

In [ ]:
import io
import requests

url = "https://cocl.us/Geospatial_data"
latlong = requests.get(url).text
df_latlong = pd.read_csv(io.StringIO(latlong))
df_latlong

In [ ]:
Canada_df = pd.merge(df,df_latlong, on ="Postal Code")
Canada_df = Canada_df.rename(columns={'Neighbourhood':'Neighborhood'})
Canada_df

In [ ]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 3

Canada_df_clustering = Canada_df.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Canada_df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Canada_df = df_latlong

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Canada_df = df_latlong.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Canada_df.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_latlong['Latitude'], df_latlong['Longitude'], df_latlong['Neighborhood'], df_latlong['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters